In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! pip install glove-python-binary

In [ ]:
from glove import Glove
glove=Glove()
glove_model=glove.load("/content/drive/MyDrive/FYP/CAA_NRC/Glove/glove.model")

In [ ]:
import pandas as pd
import numpy as np
df=pd.read_csv("/content/drive/MyDrive/FYP/CAA_NRC/CAA_NRC-preprocessed.csv")
y=pd.get_dummies(df["label"])
df

,label,new_text
0,0,muslim immigrant says that we will conquer ger...
1,1,my message to indian muslims kashmir is the sp...
2,2,normally never i abuse anyone on twitter but t...
3,1,heard someone suggesting holding anti caa nrc ...
4,2,the apex court has issued a notice to the unio...
...,...,...
3096,2,save hindus infiltrators anti national go to p...
3097,2,breaking bhimarmychief chandrashekhar azad has...
3098,0,tamil nadu cm edappadi k palaniswamy says citi...
3099,2,some more photos of protest meetings against c...


In [ ]:
maxi=-1
for i,rev in enumerate(df['new_text']):
  tokens=rev.split()
  if(len(tokens)>maxi):
    maxi=len(tokens)
print(maxi)

61


In [ ]:

import tensorflow as tf
maxlen=maxi
max_words=len(glove_model.dictionary)
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_words)
def padded_sequence(X):
  tokenizer.fit_on_texts(X)
  sequences = tokenizer.texts_to_sequences(X)
  data = pad_sequences(sequences, maxlen=maxlen)
  return data
X=padded_sequence(df["new_text"])

In [ ]:
import numpy as np
embed_matrix=np.zeros(shape=(max_words,300))
for word,i in tokenizer.word_index.items():
  if word in glove_model.dictionary:
    embed_vector=glove_model.word_vectors[glove_model.dictionary[word]]
  # word is in the vocabulary learned by the w2v model
    embed_matrix[i]=embed_vector

In [ ]:
embed_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.10990729,  0.04230337,  0.05654864, ..., -0.20832376,
        -0.03223338, -0.21452594],
       [ 0.08457289, -0.06926741, -0.00822759, ..., -0.15099157,
        -0.02485671, -0.19643595],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import Dropout,Embedding,GRU,Dense
model=Sequential()
model.add(Embedding(input_dim=max_words,output_dim=300,input_length=maxi,embeddings_initializer=Constant(embed_matrix)))
model.add(GRU(128,return_sequences=True))
model.add(GRU(128,return_sequences=False)) 
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.60))
model.add(Dense(16,activation='relu'))
model.add(Dropout(0.60))
model.add(Dense(3,activation='softmax'))
# model = Sequential()
# model.add(GRU(10,activation="tanh",dtype='float32'))
# model.add(Dense(128,activation="tanh"))
# #model.add(Dense(64,activation="sigmoid"))
# # model.add(Dense(1,activation="sigmoid"))
# model.add(Dense(3,activation="softmax"))


In [ ]:
import time

start = time.time()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1)

stop = time.time()
print(f"Training time: {stop - start}s")

Epoch 1/10
37/37 [==============================] - 6s 38ms/step - loss: 1.0826 - accuracy: 0.4215
Epoch 2/10
37/37 [==============================] - 1s 40ms/step - loss: 1.0621 - accuracy: 0.4292
Epoch 3/10
37/37 [==============================] - 2s 43ms/step - loss: 1.0081 - accuracy: 0.4688
Epoch 4/10
37/37 [==============================] - 2s 41ms/step - loss: 0.8616 - accuracy: 0.5738
Epoch 5/10
37/37 [==============================] - 2s 42ms/step - loss: 0.7319 - accuracy: 0.6826
Epoch 6/10
37/37 [==============================] - 2s 42ms/step - loss: 0.5834 - accuracy: 0.7660
Epoch 7/10
37/37 [==============================] - 2s 40ms/step - loss: 0.4671 - accuracy: 0.8163
Epoch 8/10
37/37 [==============================] - 2s 40ms/step - loss: 0.3784 - accuracy: 0.8645
Epoch 9/10
37/37 [==============================] - 2s 46ms/step - loss: 0.3426 - accuracy: 0.8822
Epoch 10/10
37/37 [==============================] - 2s 43ms/step - loss: 0.3082 - accuracy: 0.9058
Training 

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
def display_metrics(y_test,y_pred):
    print(accuracy_score(y_test, y_pred))
    print(f1_score(y_test, y_pred, average="macro"))
    print(precision_score(y_test, y_pred, average="macro"))
    print(recall_score(y_test, y_pred, average="macro"))


In [ ]:
import numpy as np
y_pred=model.predict(X_test)
y_test=y_test.to_numpy()
l=[np.argmax(i) for i in y_pred]
M=[np.argmax(i) for i in y_test]
display_metrics(M,l)

0.645618556701031
0.6340891671682443
0.6389776808051704
0.6322481725696756
